In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/covid19-tweets/covid19_tweets.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

# Covid 19 Tweet Sentiment Analysis

## Process Datasets

In [ ]:
# Drop Columns
drop_columns = [
    'user_name',
    'user_description',
    'user_created',
    'user_followers',
    'user_friends',
    'user_favourites',
    'user_verified',
    'hashtags',
    'source',
    'is_retweet'
]

df.drop(columns=drop_columns, inplace=True)
df.head()

## Basic Tweet Processing
* Make lower case
* Remove Mentions
* Remove URLs
* Remove Stopwords (Common words without much meaning)
* Remove punctuations 

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist

from textblob import TextBlob
from textblob import Word


stop_words = set(stopwords.words('english'))

def tweet_scrubber(tweet):
   
    # Make tweets all lower case
    tweet = tweet.lower()
#     # Remove Mentions
    tweet = re.sub(r'@[A-Za-z0-9_]+','',tweet)
#     # Remove URLs
#     tweet = re.sub(r"http\S+", "", tweet)
#     # Remove Numbers
#     tweet = re.sub(r"[0-9]*", "", tweet)
#     # Remove Special Characters, and Punctuations
#     tweet = re.sub(r"(”|“|-|\+|`|#|,|;|\|)*", "", tweet)
#     tweet = re.sub(r"&amp", "", tweet)
    
    to_remove = r'\d+|http?\S+|[^A-Za-z0-9]+'
    tweet = re.sub(to_remove, " ", tweet)
    return tweet
        

def tokenization_and_stem(tweet, stem=False, lemmatize=False):
        
    filtered_tweet = []
    words = word_tokenize(tweet) 

    # Remove stopwords and stem
    for word in words:
        if not word in stop_words:
            if lemmatize:
                filtered_tweet.append(Word(word).lemmatize())
            else:
                filtered_tweet.append(word)
            
    return filtered_tweet

In [ ]:
df.text = df.text.apply(lambda x: tokenization_and_stem(tweet_scrubber(x), lemmatize=True))

In [ ]:
df.text.head()

## Sentiment Analysis

In [ ]:
# Function to perform sentitment analysis on trump and biden dataframes
def sentiment_analysis(df):
    
    # Determine polarity and subjectivity
    df['Polarity'] = df['text'].apply(lambda x: TextBlob(' '.join(x)).sentiment.polarity)
    df['Subjectivity'] = df['text'].apply(lambda x: TextBlob(' '.join(x)).sentiment.subjectivity)
    
    # Classify overall sentiment
    df.loc[df.Polarity > 0,'Sentiment'] = 'positive'
    df.loc[df.Polarity == 0,'Sentiment'] = 'neutral'
    df.loc[df.Polarity < 0,'Sentiment'] = 'negative'
    
    return df[['text','Polarity','Subjectivity','Sentiment']].head()

In [ ]:
sentiment_analysis(df)

In [ ]:
df['Sentiment'].value_counts()